In [3]:
import os
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import Tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain import LLMMathChain

In [5]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document

load_dotenv()

True

In [6]:
class PatientRAG:
    def __init__(self, patients_directory: str):
        self.patients_directory = patients_directory
        self.embeddings = OpenAIEmbeddings()
        self.vector_store = self.load_patient_data()

    def load_patient_data(self):
        documents = []
        for filename in os.listdir(self.patients_directory):
            if filename.endswith('.txt'):
                file_path = os.path.join(self.patients_directory, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    documents.append(Document(page_content=content, metadata={"source": filename}))
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(documents)
        return Chroma.from_documents(splits, self.embeddings)

    def query(self, question: str) -> str:
    # Primero, intentamos buscar por nombre de archivo
        if question.lower().startswith("paciente"):
            file_name = f"{question.lower()}.txt"
            file_path = os.path.join(self.patients_directory, file_name)
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    return f"Información del {question}:\n{file.read()}"
        
        # Si no se encuentra por nombre de archivo, usamos el retriever
        retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        qa_chain = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(temperature=0, model_name="gpt-4"),
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
        result = qa_chain({"query": question})
        
        if not result['result']:
            return f"No se encontró información para la consulta: {question}"
        
        return f"Respuesta: {result['result']}\nFuentes: {[doc.metadata['source'] for doc in result['source_documents']]}"

In [7]:
class DoctorAgent:
    def diagnose(self, symptoms: str) -> str:
        return f"Basado en los síntomas '{symptoms}', el diagnóstico preliminar es..."

class NurseAgent:
    def check_vitals(self, patient_name: str) -> str:
        return f"Los signos vitales de {patient_name} son normales."

class AdminAgent:
    def schedule_appointment(self, patient_name: str, date: str) -> str:
        return f"Cita agendada para {patient_name} el {date}."


In [8]:
from langchain.tools import Tool, StructuredTool
from pydantic import BaseModel, Field

class HospitalTools:
    def __init__(self, rag_system: PatientRAG):
        self.rag_system = rag_system
        self.doctor = DoctorAgent()
        self.nurse = NurseAgent()
        self.admin = AdminAgent()

    def list_patients(self) -> str:
        return ', '.join([f.split('.')[0] for f in os.listdir(self.rag_system.patients_directory) if f.endswith('.txt')])

    def get_tools(self):
        class DiagnoseInput(BaseModel):
            symptoms: str = Field(description="Síntomas del paciente")

        class CheckVitalsInput(BaseModel):
            patient_name: str = Field(description="Nombre del paciente")

        class ScheduleAppointmentInput(BaseModel):
            patient_name: str = Field(description="Nombre del paciente")
            date: str = Field(description="Fecha de la cita")

        class QueryInput(BaseModel):
            question: str = Field(description="Pregunta sobre el paciente")

        return [
            StructuredTool.from_function(
                func=self.rag_system.query,
                name="consulta_paciente",
                description="Útil para buscar información específica sobre un paciente.",
                args_schema=QueryInput
            ),
            StructuredTool.from_function(
                func=self.doctor.diagnose,
                name="diagnostico",
                description="Útil para obtener un diagnóstico preliminar basado en síntomas.",
                args_schema=DiagnoseInput
            ),
            StructuredTool.from_function(
                func=self.nurse.check_vitals,
                name="verificar_signos_vitales",
                description="Útil para verificar los signos vitales de un paciente.",
                args_schema=CheckVitalsInput
            ),
            StructuredTool.from_function(
                func=self.admin.schedule_appointment,
                name="agendar_cita",
                description="Útil para agendar una cita para un paciente.",
                args_schema=ScheduleAppointmentInput
            ),
            StructuredTool.from_function(
                func=self.list_patients,
                name="listar_pacientes",
                description="Útil para obtener una lista de todos los pacientes disponibles."
            )
        ]

In [9]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

class HospitalAgent:
    def __init__(self, tools):
        self.llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
        prompt = hub.pull("hwchase17/openai-functions-agent")
        self.agent = create_openai_functions_agent(self.llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=self.agent, tools=tools, verbose=True)

    def run(self, query: str) -> Dict[str, str]:
        result = self.agent_executor.invoke({"input": query})
        tool_used = result.get('intermediate_steps', [])
        if tool_used:
            tool_name = tool_used[0][0].tool
        else:
            tool_name = "No se utilizó ninguna herramienta específica"
        return {
            "output": result["output"],
            "tool_used": tool_name
        }

In [14]:
import logging

logging.basicConfig(level=logging.INFO)

class HospitalApplication:
    def __init__(self, patients_directory: str):
        self.patients_directory = patients_directory
        rag_system = PatientRAG(patients_directory)
        hospital_tools = HospitalTools(rag_system)
        self.agent = HospitalAgent(hospital_tools.get_tools())

    def run_interaction(self):
        print("Bienvenido al Sistema de Consulta de Pacientes del Hospital")
        while True:
            query = input("\nPor favor, ingrese su consulta (o 'salir' para terminar): ")
            if query.lower() == 'salir':
                print("Gracias por usar el Sistema de Consulta de Pacientes del Hospital. ¡Hasta luego!")
                break
            try:
                logging.info(f"Consulta recibida: {query}")
                response = self.agent.run(query)
                logging.info(f"Respuesta generada: {response['output']}")
                logging.info(f"Herramienta utilizada: {response['tool_used']}")
                print("\nRespuesta:", response['output'])
                print(f"Herramienta utilizada: {response['tool_used']}")
            except Exception as e:
                logging.error(f"Error ocurrido: {str(e)}")
                print(f"\nOcurrió un error: {str(e)}")
                print("Por favor, intente reformular su consulta.")

In [15]:
app = HospitalApplication("../data/pacientes")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [16]:
app.run_interaction()

Bienvenido al Sistema de Consulta de Pacientes del Hospital


INFO:root:Consulta recibida: lista de pacientes




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `listar_pacientes` with `{}`


paciente2, paciente3, paciente1, paciente4, paciente5

...


Invoking: `consulta_paciente` with `{'question': '¿Cuál es el tratamiento actual de Juan Perez?'}`




/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/hospitalmultiagentllmlangchain-kpg3edPC-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Respuesta: El tratamiento actual de Juan Pérez García es el inicio de tratamiento con Enalapril 10 mg, 1 tableta diaria. Además, se le ha recomendado una dieta baja en sodio y un aumento de actividad física. Se sugiere un seguimiento mensual para monitorear su presión arterial y ajustar el tratamiento si es necesario.
Fuentes: ['paciente1.txt', 'paciente1.txt', 'paciente2.txt']

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Respuesta generada: El tratamiento actual de Juan Pérez García es el inicio de tratamiento con Enalapril 10 mg, 1 tableta diaria. Además, se le ha recomendado una dieta baja en sodio y un aumento de actividad física. Se sugiere un seguimiento mensual para monitorear su presión arterial y ajustar el tratamiento si es necesario.
INFO:root:Herramienta utilizada: No se utilizó ninguna herramienta específica


El tratamiento actual de Juan Pérez García es el inicio de tratamiento con Enalapril 10 mg, 1 tableta diaria. Además, se le ha recomendado una dieta baja en sodio y un aumento de actividad física. Se sugiere un seguimiento mensual para monitorear su presión arterial y ajustar el tratamiento si es necesario.

> Finished chain.

Respuesta: El tratamiento actual de Juan Pérez García es el inicio de tratamiento con Enalapril 10 mg, 1 tableta diaria. Además, se le ha recomendado una dieta baja en sodio y un aumento de actividad física. Se sugiere un seguimiento mensual para monitorear su presión arterial y ajustar el tratamiento si es necesario.
Herramienta utilizada: No se utilizó ninguna herramienta específica
Gracias por usar el Sistema de Consulta de Pacientes del Hospital. ¡Hasta luego!
